In [14]:
from selenium import webdriver
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
import re
import time
from datetime import date, timedelta

In [15]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

In [16]:
root = 'https://rotogrinders.com'

In [17]:
def get_base_page(date, Fanduel=False):
    if Fanduel:
        url = "https://rotogrinders.com/resultsdb/site/fanduel/date/{}/sport/nba".format(date)
        return driver.get(url)
    else:
        url = "https://rotogrinders.com/resultsdb/site/draftkings/date/{}/sport/nba".format(date)
        return driver.get(url)
def click_ownership_page():
    for elt in driver.find_elements_by_class_name("ant-tabs-tab"):
        if "Ownership" in elt.text:
            elt.click()
            return
    raise Exception('no button')
def click_summary_page():
    for elt in driver.find_elements_by_class_name("ant-tabs-tab"):
        if "Summary" in elt.text:
            elt.click()
            return
    raise Exception('no button')
def click_contests_page():
    for elt in driver.find_elements_by_class_name("ant-tabs-tab"):
        if "Contests" in elt.text:
            elt.click()
            return
    raise Exception('no button')
def click_right():
    driver.find_elements_by_css_selector('.ant-btn.ant-btn-icon-only')[1].click()
def get_winning_results():
    soup=BeautifulSoup(driver.page_source,'html.parser')
    tables = soup.findAll("table")
    table=pd.read_html(str(tables[0]))[0]
    for i in range(200):
        try:
            click_right()
        except:
            break
        time.sleep(.02)
        soup=BeautifulSoup(driver.page_source,'html.parser')
        tables = soup.findAll("table")
        a =pd.read_html(str(tables[0]))[0]
        table = table.append(a)
    return table
def get_actual_points():
    time.sleep(3)
    click_ownership_page()
    time.sleep(1)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    tables = soup.findAll("table")
    df = pd.read_html(str(tables[2]))[0]
    return df
def get_contests(date):
    output = []
    time.sleep(.1)
    try:
        click_contests_page()
    except:
        return output
    time.sleep(.1)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    tables = soup.findAll("table")
    #print(tables)
    index = 0 if len(tables) == 1 else 2
    rows = tables[index].find("tbody").find_all("tr")
    records = pd.read_html(str(tables[index]))[0].to_dict(orient='records')
    for i in range(len(rows)):
        #print(rows[i].find_all('a', href=True))
        if float(records[i]['Top Prize'][1:].replace(",","")) > 4000 and float(records[i]['Buy In'][1:].replace(",","")) < 30 and float(records[i]['Max Entries']) >= 10: 
            print(root+rows[i].find_all('a', href=True)[0]['href'])
            driver.get(root+rows[i].find_all('a', href=True)[0]['href'])
            time.sleep(10)
            print(records[i])
            df = get_winning_results()
            records[i]['PAYOUTS']=df
            output.append(records[i])
    return output
def projections(date_string):
    output = {}
    url = 'https://rotogrinders.com/projected-stats/nba-player?site=draftkings&date={}'.format(date_string)
    driver.get(url)
    #print('hello')
    soup=BeautifulSoup(driver.page_source,'html.parser')
    mydivs = soup.findAll("div", {"class": "rgt-col"})
    try:
        floor = [float(a.text) for a in mydivs[-3].findAll("div")[1:]]
        ceil = [float(a.text) for a in mydivs[-4].findAll("div")[1:]]
        points = [float(a.text) for a in mydivs[-2].findAll("div")[1:]]
        name = [a.text for a in mydivs[0].findAll("div")[1:]]
        salary = [float(a.text[1:-1])*1000 for a in mydivs[1].findAll("div")[1:]]
        team = [a.text for a in mydivs[2].findAll("div")[1:]]
    except:
        return False
    for i in range(len(floor)):
        output[name[i]] = {'ceil':ceil[i], 'floor': floor[i], 'points':points[i], 'salary':salary[i], 'team':team[i]}
        #print(name[i], ceil[i], floor[i], points[i])
    return output
def scrape_data(date):
    try:
        get_base_page(date,Fanduel=False)
    except:
        return False
    time.sleep(8)
    try:
        actual_points = get_actual_points()
    except:
        return False
    contests = get_contests(date)
    projection = projections(date)
    return {"ACTUAL_POINTS": actual_points, "CONTESTS": contests, "PROJECTIONS": projection}
def scrape_all_data(Sdate = date(2017,9,1),Edate = date.today()):
    huge_data = {}
    sdate = Sdate   
    edate = Edate
    #driver = webdriver.Chrome('/usr/local/bin/chromedriver')

    delta = edate - sdate       

    for i in range(delta.days + 1):
        day = sdate + timedelta(days=i)
        
        text = day.strftime("%Y-%m-%d")
        print(text)
        #huge_data[text] = scrape_data(text)
        result = scrape_data(text)
        if result != False:
            huge_data[text] = result
    return huge_data

In [23]:
import pickle
for i in range(9,13):
    print(i)
    if i != 12:
        result1 = scrape_all_data(date(2019,i,1), date(2019,i,1))
    else:
        result1 = scrape_all_data(date(2019,12,1), date(2020,1,1))
    filehandler = open("2019-{}.pkl".format(i),"wb")
    pickle.dump(result1,filehandler)
    filehandler.close()

9
2019-09-01
10
2019-10-01
11
2019-11-01
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-11-01/sport/nba/slate/5dbcb971730ea13a3f448f0c/contest/5dbcb971615e603a4270fe2e
{'Name': "NBA $400K Excellent 8's [$100K to 1st]", 'Link': 'Contest', 'Prize Pool': '$400,000', 'Buy In': '$8', 'Top Prize': '$100,000', 'Max Entries': 150, 'Entries': 58824, 'Cash Line': 280.0, 'Winner': 'SunSucks', 'Winning Score': 387.0}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-11-01/sport/nba/slate/5dbcb971730ea13a3f448f0c/contest/5dbcba384639903a47c1d487
{'Name': "NBA $50K Excellent 8's [$10K to 1st]", 'Link': 'Contest', 'Prize Pool': '$50,000', 'Buy In': '$8', 'Top Prize': '$10,000', 'Max Entries': 150, 'Entries': 7353, 'Cash Line': 280.75, 'Winner': 'MasonMiller12', 'Winning Score': 373.75}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-11-01/sport/nba/slate/5dbcb971730ea13a3f448f0c/contest/5dbd9557c320ad40896ab17e
{'Name': 'NBA $40K Step-Back Fadeaway [$10K to 1st]'

{'Name': 'NBA $400K Friday Fadeaway [$100K to 1st]', 'Link': 'Contest', 'Prize Pool': '$400,000', 'Buy In': '$15', 'Top Prize': '$100,000', 'Max Entries': 150, 'Entries': 31373, 'Cash Line': 305.75, 'Winner': 'Dinkpiece', 'Winning Score': 387.0}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-06/sport/nba/slate/5deaebfb84031903d3e2bafd/contest/5deaec7099ad8703cc0ff2ed
{'Name': 'NBA $100K Four Point Play [20 Entry Max, 2x Min Cash]', 'Link': 'Contest', 'Prize Pool': '$100,000', 'Buy In': '$4', 'Top Prize': '$10,000', 'Max Entries': 20, 'Entries': 29727, 'Cash Line': 298.5, 'Winner': 'doddy8787', 'Winning Score': 400.5}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-06/sport/nba/slate/5deaebfb84031903d3e2bafd/contest/5deaec7099ad8703cc0ff43c
{'Name': 'NBA $60K Friday Pull Up Jumper [$15K to 1st]', 'Link': 'Contest', 'Prize Pool': '$60,000', 'Buy In': '$10', 'Top Prize': '$15,000', 'Max Entries': 150, 'Entries': 7059, 'Cash Line': 305.75, 'Winner': 'GoMaiz

https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-11/sport/nba/slate/5df183d87c57fa2db0cb16c2/contest/5df1845904e2b42dbe06393d
{'Name': 'NBA $100K Four Point Play [20 Entry Max, 2x Min Cash]', 'Link': 'Contest', 'Prize Pool': '$100,000', 'Buy In': '$4', 'Top Prize': '$10,000', 'Max Entries': 20, 'Entries': 29727, 'Cash Line': 296.0, 'Winner': 'ctycksen', 'Winning Score': 382.0}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-11/sport/nba/slate/5df183d87c57fa2db0cb16c2/contest/5df184577df845312cb288a9
{'Name': 'NBA $85K Full Court Press [$15K to 1st]', 'Link': 'Contest', 'Prize Pool': '$85,000', 'Buy In': '$20', 'Top Prize': '$15,000', 'Max Entries': 150, 'Entries': 5001, 'Cash Line': 298.0, 'Winner': 'rrich68', 'Winning Score': 364.5}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-11/sport/nba/slate/5df183d87c57fa2db0cb16c2/contest/5df26df70821c8359a179b24
{'Name': 'NBA $25K Fadeaway [$5K to 1st]', 'Link': 'Contest', 'Prize Pool': '$25,000

{'Name': 'NBA $25K Fadeaway [$5K to 1st]', 'Link': 'Contest', 'Prize Pool': '$25,000', 'Buy In': '$15', 'Top Prize': '$5,000', 'Max Entries': 58, 'Entries': 1961, 'Cash Line': 313.75, 'Winner': 'grtwhitehoop', 'Winning Score': 353.75}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-16/sport/nba/slate/5df81af6d1540b5727af4ad4/contest/5df8f7bb0c6f395c5830cd12
{'Name': 'NBA $25K Zone [$5K to 1st]', 'Link': 'Contest', 'Prize Pool': '$25,000', 'Buy In': '$5', 'Top Prize': '$5,000', 'Max Entries': 150, 'Entries': 5946, 'Cash Line': 315.75, 'Winner': 'Talapia69', 'Winning Score': 365.0}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-16/sport/nba/slate/5df81af6d1540b5727af4ad4/contest/5df8f745a1473c5c629b1e17
{'Name': 'NBA $25K Jab Step [$5K to 1st]', 'Link': 'Contest', 'Prize Pool': '$25,000', 'Buy In': '$6', 'Top Prize': '$5,000', 'Max Entries': 147, 'Entries': 4902, 'Cash Line': 315.0, 'Winner': 'dctat33', 'Winning Score': 363.75}
2019-12-17
https://rotogrin

https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-19/sport/nba/slate/5dfc16fadebbac7dbdfa46ab/contest/5dfcecd87cb660060349951d
{'Name': 'NBA $25K Pull Up Jumper [$5K to 1st]', 'Link': 'Contest', 'Prize Pool': '$25,000', 'Buy In': '$10', 'Top Prize': '$5,000', 'Max Entries': 88, 'Entries': 2942, 'Cash Line': 278.25, 'Winner': 'nickrambo', 'Winning Score': 335.75}
2019-12-20
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-20/sport/nba/slate/5dfd60f87cb66006034acc87/contest/5dfd60f9dd0a0905fbffa7ea
{'Name': 'NBA $500K Holiday Warm Up [Bonus $25K in Tix, 100K to 1st]', 'Link': 'Contest', 'Prize Pool': '$525,000', 'Buy In': '$25', 'Top Prize': '$100,000', 'Max Entries': 150, 'Entries': 23530, 'Cash Line': 265.25, 'Winner': 'Awesemo', 'Winning Score': 332.5}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-20/sport/nba/slate/5dfd60f87cb66006034acc87/contest/5dfd614d7cb66006034ad04f
{'Name': 'NBA $200K Zone [$50K to 1st]', 'Link': 'Contest', 'Prize

{'Name': 'NBA $125K Four Point Play [20 Entry Max, 2x Min Cash]', 'Link': 'Contest', 'Prize Pool': '$125,000', 'Buy In': '$4', 'Top Prize': '$12,000', 'Max Entries': 20, 'Entries': 37159, 'Cash Line': 246.0, 'Winner': 'MicahJ', 'Winning Score': 351.0}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-26/sport/nba/slate/5e0549ec68ed187816123677/contest/5e062c2454d7332eafd22224
{'Name': 'NBA $40K Jab Step [$10K to 1st]', 'Link': 'Contest', 'Prize Pool': '$40,000', 'Buy In': '$6', 'Top Prize': '$10,000', 'Max Entries': 150, 'Entries': 7844, 'Cash Line': 254.5, 'Winner': 'MicahJ', 'Winning Score': 351.0}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-26/sport/nba/slate/5e0549ec68ed187816123677/contest/5e062beb54d7332eafd1e930
{'Name': 'NBA $25K Pull Up Jumper [$5K to 1st]', 'Link': 'Contest', 'Prize Pool': '$25,000', 'Buy In': '$10', 'Top Prize': '$5,000', 'Max Entries': 88, 'Entries': 2942, 'Cash Line': 253.5, 'Winner': 'rynpak', 'Winning Score': 328.25}
htt

https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-30/sport/nba/slate/5e0a90076e967d3153bcf6a4/contest/5e0b71b5d527bc30bcec1d18
{'Name': 'NBA $25K Zone [$5K to 1st]', 'Link': 'Contest', 'Prize Pool': '$25,000', 'Buy In': '$5', 'Top Prize': '$5,000', 'Max Entries': 150, 'Entries': 5946, 'Cash Line': 309.25, 'Winner': 'bric75', 'Winning Score': 363.0}
2019-12-31
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-31/sport/nba/slate/5e0be18fd527bc30bcf14cea/contest/5e0be191d527bc30bcf14fe8
{'Name': 'NBA $500K Decade Ender [$100K to 1st!]', 'Link': 'Contest', 'Prize Pool': '$500,000', 'Buy In': '$19', 'Top Prize': '$100,000', 'Max Entries': 150, 'Entries': 30960, 'Cash Line': 286.25, 'Winner': 'FTAKJ | FTAKJ', 'Winning Score': 332.0}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-12-31/sport/nba/slate/5e0be18fd527bc30bcf14cea/contest/5e0be21cd527bc30bcf1cae8
{'Name': 'NBA $125K Four Point Play [20 Entry Max, 2x Min Cash]', 'Link': 'Contest', 'Prize Po

In [52]:


filehandler = open("2018_season_second_half.pkl","wb")
pickle.dump(result1,filehandler)
filehandler.close()

In [53]:
projections("2019-12-01")

{'Tyus Jones': {'ceil': 47.41,
  'floor': 11.96,
  'points': 29.52,
  'salary': 3100.0,
  'team': 'MEM'},
 'Isaac Bonga': {'ceil': 27.86,
  'floor': 3.05,
  'points': 14.05,
  'salary': 1500.0,
  'team': 'WAS'},
 'Jalen Brunson': {'ceil': 23.05,
  'floor': 2.82,
  'points': 11.53,
  'salary': 1500.0,
  'team': 'DAL'},
 'Brandon Clarke': {'ceil': 65.14,
  'floor': 10.46,
  'points': 35.23,
  'salary': 4800.0,
  'team': 'MEM'},
 'Dillon Brooks': {'ceil': 54.47,
  'floor': 13.03,
  'points': 29.54,
  'salary': 4500.0,
  'team': 'MEM'},
 'Kevin Knox': {'ceil': 38.74,
  'floor': 6.22,
  'points': 21.46,
  'salary': 3300.0,
  'team': 'NYK'},
 'Jaren Jackson': {'ceil': 62.32,
  'floor': 13.3,
  'points': 35.47,
  'salary': 5600.0,
  'team': 'MEM'},
 'Solomon Hill': {'ceil': 37.58,
  'floor': 4.81,
  'points': 18.8,
  'salary': 3000.0,
  'team': 'MEM'},
 'DeAndre Jordan': {'ceil': 49.03,
  'floor': 10.01,
  'points': 29.27,
  'salary': 4700.0,
  'team': 'BKN'},
 'Jae Crowder': {'ceil': 50.59,


In [26]:
result1 = scrape_all_data(date(2019,10,22), date(2019,11,1))

filehandler = open("2019-10.pkl".format(i),"wb")
pickle.dump(result1,filehandler)
filehandler.close()

2019-10-22
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-10-22/sport/nba/slate/5daf98650e29a96829bf82d8/contest/5daf98650e29a96829bf8329
{'Name': 'NBA $1M Opening Tip Off [$200K to 1st]', 'Link': 'Contest', 'Prize Pool': '$1,000,000', 'Buy In': '$10', 'Top Prize': '$200,000', 'Max Entries': 150, 'Entries': 110075, 'Cash Line': 292.25, 'Winner': 'jqspeedy82 | Deje8352 | Ogre1234', 'Winning Score': 366.0}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-10-22/sport/nba/slate/5daf98650e29a96829bf82d8/contest/5daf9900699c7468368a8920
{'Name': 'NBA $100K Sharpshooter [20 Entry Max]', 'Link': 'Contest', 'Prize Pool': '$100,000', 'Buy In': '$3', 'Top Prize': '$10,000', 'Max Entries': 20, 'Entries': 39636, 'Cash Line': 288.75, 'Winner': 'Diggey13 | ZEUSMAN1711', 'Winning Score': 366.0}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-10-22/sport/nba/slate/5daf98650e29a96829bf82d8/contest/5daf9904eec2a968288c841f
{'Name': 'NBA $100K mini-MAX [150 Entry Max

https://rotogrinders.com/resultsdb/site/draftkings/date/2019-10-26/sport/nba/slate/5db4d04fa549a10a1b973210/contest/5db4d11ba549a10a1b981a35
{'Name': 'NBA $50K mini-MAX [150 Entry Max]', 'Link': 'Contest', 'Prize Pool': '$50,000', 'Buy In': '$2', 'Top Prize': '$5,000', 'Max Entries': 150, 'Entries': 29727, 'Cash Line': 297.0, 'Winner': 'Hpotts', 'Winning Score': 387.75}
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-10-26/sport/nba/slate/5db4d04fa549a10a1b973210/contest/5db5de1d91e3b8142d43d458
{'Name': 'NBA $40K Step-Back Fadeaway [$10K to 1st]', 'Link': 'Contest', 'Prize Pool': '$40,000', 'Buy In': '$15', 'Top Prize': '$10,000', 'Max Entries': 94, 'Entries': 3138, 'Cash Line': 298.75, 'Winner': 'dtown_uptown32', 'Winning Score': 378.0}
2019-10-27
https://rotogrinders.com/resultsdb/site/draftkings/date/2019-10-27/sport/nba/slate/5db70a15f625841c65f45b97/contest/5db70a88f625841c65f55a37
{'Name': "NBA $200K Excellent 8's [$50K to 1st]", 'Link': 'Contest', 'Prize Pool': '$2